In [ ]:
import random

# ==== RATING CALCULATION ====

def compute_rating(xGF, TSV, HIWR):
    return (xGF * 0.35) + (TSV * 0.45) + (HIWR * 0.2)


# ==== GAME & SERIES SIMULATION ====

def game_win_prob(team_rating, opp_rating):
    # Handle case where both ratings are 0 to prevent ZeroDivisionError
    if (team_rating + opp_rating) == 0:
        return 0.5 # Default to 50/50 if no rating information
    return (team_rating) / (team_rating + opp_rating)

def simulate_game_100_times(team_rating, opp_rating):
    team_wins = 0
    opp_wins = 0
    for _ in range(100): # Simulate each game 100 times
        prob = game_win_prob(team_rating, opp_rating)
        if random.random() < prob:
            team_wins += 1
        else:
            opp_wins += 1
    return team_wins > opp_wins # Return True if team wins more simulations, False otherwise

def get_home_games(round_name, home_team_is_A):
    formats = {
        "quarters":       [True, True, False, False, True, False, True],
        "semis":          [True, True, False, False, True, False, True],
        "finals":         [True, True, False, False, True, False, True],
        "stanley_cup":    [True, True, False, False, True, False, True],
    }
    base = formats[round_name]
    return base if home_team_is_A else [not g for g in base]

def simulate_series(team_A, team_B, round_name):
    series_format = {
        "quarters":     (4, 7),
        "semisl":       (4, 7),
        "finals":       (4, 7),
        "stanley_cup":  (4, 7)
    }[round_name]

    wins_needed, max_games = series_format

    # HOME FIELD LOGIC
    if round_name == "world_series":
        # For World Series, the home field advantage is determined by the team with the better regular season record.
        # If records are tied, then by seed.
        if team_A["wins"] != team_B["wins"]:
            home_team_is_A = team_A["wins"] > team_B["wins"]
        else:
            home_team_is_A = team_A["seed"] < team_B["seed"]
    else:
        home_team_is_A = team_A["seed"] < team_B["seed"]

    home_games = get_home_games(round_name, home_team_is_A)
    home_team_in_series = team_A if home_team_is_A else team_B
    away_team_in_series = team_B if home_team_is_A else team_A


    team_A_wins = 0
    team_B_wins = 0

    print(f"\n🎯 {round_name.upper()} SERIES: {team_A['name']} (Seed {team_A['seed']}) vs {team_B['name']} (Seed {team_B['seed']})")
    print(f"🏟 Home field advantage: {home_team_in_series['name']}")

    for i in range(max_games):
        # Ensure 'i' is within the bounds of 'home_games' list
        if i >= len(home_games):
            break

        is_home_for_A = home_games[i]
        
        # Determine which team is home for this specific game
        current_home_team = team_A if is_home_for_A else team_B

        winner_A = simulate_game_100_times(team_A['rating'], team_B['rating'], is_home_for_A)

        if winner_A:
            team_A_wins += 1
            print(f"Game {i+1} (Home: {current_home_team['name']}): {team_A['name']} win.")
        else:
            team_B_wins += 1
            print(f"Game {i+1} (Home: {current_home_team['name']}): {team_B['name']} win.")

        if team_A_wins == wins_needed:
            print(f"🏆 {team_A['name']} win the {round_name} series {team_A_wins}-{team_B_wins}")
            return team_A
        elif team_B_wins == wins_needed:
            print(f"🏆 {team_B['name']} win the {round_name} series {team_B_wins}-{team_A_wins}")
            return team_B


# ==== POSTSEASON BRACKET SIMULATION ====

def simulate_postseason(teams_AL, teams_NL):
    print("\n⚾ SIMULATING MLB POSTSEASON BRACKET\n")

    # Wild Card Round
    ALWC1 = simulate_series(teams_AL[3], teams_AL[6], "wildcard")
    ALWC2 = simulate_series(teams_AL[4], teams_AL[5], "wildcard")
    NLWC1 = simulate_series(teams_NL[3], teams_NL[6], "wildcard")
    NLWC2 = simulate_series(teams_NL[4], teams_NL[5], "wildcard")

    # Division Series
    ALDS1 = simulate_series(teams_AL[1], ALWC2, "divisional")
    ALDS2 = simulate_series(teams_AL[2], ALWC1, "divisional")
    NLDS1 = simulate_series(teams_NL[1], NLWC2, "divisional")
    NLDS2 = simulate_series(teams_NL[2], NLWC1, "divisional")

    # League Championship
    ALCS = simulate_series(ALDS1, ALDS2, "lcs")
    NLCS = simulate_series(NLDS1, NLDS2, "lcs")

    # World Series
    WS_winner = simulate_series(ALCS, NLCS, "world_series")
    print(f"\n🏁 WORLD SERIES CHAMPION: {WS_winner['name']} 🏆\n")


# ==== TEAM DATA ====

def create_team(name, wins, losses, runs_scored, runs_allowed, on_base, slugging, seed):
    return {
        "name": name,
        "wins": wins,
        "losses": losses,
        "runs_scored": runs_scored,
        "runs_allowed": runs_allowed,
        "on_base": on_base,
        "slugging": slugging,
        "seed": seed,
        "rating": compute_rating(wins, losses, runs_scored, runs_allowed, on_base, slugging)
    }

teams_AL = [
    None, # 0 index is unused, keeping for 1-based seed indexing
    create_team("Toronto Blue Jays", 94, 68, 798, 721, 0.333, 0.427, 1),
    create_team("Seattle Mariners", 90, 72, 766, 694, 0.320, 0.420, 2),
    create_team("Cleveland Guardians", 88, 74, 643, 649, 0.296, 0.373, 3),
    create_team("New York Yankees", 94, 68, 849, 685, 0.332, 0.455, 4),
    create_team("Boston Red Sox", 89, 73, 786, 676, 0.324, 0.421, 5),
    create_team("Detroit Tigers", 87, 75, 758, 691, 0.316, 0.413, 6),
]

teams_NL = [
    None, # 0 index is unused, keeping for 1-based seed indexing
    create_team("Milwaukee Brewers", 97, 65, 806, 634, 0.332, 0.420, 1),
    create_team("Philadelphia Phillies", 96, 66, 778, 648, 0.328, 0.431, 2),
    create_team("Los Angeles Dodgers", 93, 69, 825, 683, 0.327, 0.441, 3),
    create_team("Chicago Cubs", 92, 70, 793, 649, 0.320, 0.430, 4),
    create_team("San Diego Padres", 90, 72, 702, 621, 0.321, 0.390, 5),
    create_team("Cincinnati Reds", 83, 79, 716, 681, 0.315, 0.391, 6),
]

simulate_postseason(teams_AL, teams_NL)